In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.impute import KNNImputer

In [2]:
df1 = pd.read_csv('COHORT_1_data.csv',usecols=['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])
df2 = pd.read_csv('COHORT_2_data.csv',usecols=['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])
df3 = pd.read_csv('COHORT_3_data.csv',usecols=['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])
df4 = pd.read_csv('COHORT_4_data.csv',usecols=['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])

In [3]:
df3.columns

Index(['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'],
      dtype='object')

In [4]:
# Define a function to fill missing values and apply KNN imputation
def fill_missing_and_impute(df):
    # Group DataFrame by 'PATNO' and fill missing values with median within each group
    grouped_df = df.groupby('PATNO').transform(lambda x: x.fillna(x.median()))

    # Add 'PATNO' column back to the filled DataFrame
    df_filled = pd.concat([df['PATNO'], grouped_df], axis=1)

    # Calculate the mean of each feature after filling missing values
    grouped_mean_df = df_filled.groupby('PATNO').mean().reset_index()

    # Apply KNN imputation to handle remaining missing values
    knn = KNNImputer()
    grouped_mean_df = pd.DataFrame(knn.fit_transform(grouped_mean_df), columns=grouped_mean_df.columns)
    
    return grouped_mean_df

# Apply the function to all four dataframes
df1_filled = fill_missing_and_impute(df1)
df2_filled = fill_missing_and_impute(df2)
df3_filled = fill_missing_and_impute(df3)
df4_filled = fill_missing_and_impute(df4)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\A

In [5]:
f = pd.concat([df1_filled,df2_filled,df3_filled,df4_filled])

In [6]:
le = LabelEncoder()
f['COHORT'] = le.fit_transform(f['COHORT'])

In [7]:
f.sample(10)

,PATNO,SDMTOTAL,STAI_TOT,SFT_TOT,SCOPA_AUT_TOT,REMSLEEP_TOT,QUIP_A,QUIP_B,QUIP_C,QUIP_D,...,LNS_TOT,HVLT_TOT_Recall,HVLT_DCR_REC,HVLT_RETENTION,GDS_TOT,GDS_Depressed,ESS_TOT,ESS_Sleepy,BJLOT_TOT,COHORT
632,100445.0,42.000000,88.000000,22.500000,20.500000,5.500000,0.0,0.000000,0.000000,0.0,...,8.500000,31.500000,12.000000,0.958333,10.500000,1.000000,6.500000,0.000000,14.000000,3
63,3258.0,47.782143,52.232143,49.835714,4.375000,1.832143,0.0,0.028571,0.000000,0.2,...,10.157143,25.796429,11.325000,0.908546,0.385714,0.025000,5.825000,0.100000,11.407143,1
227,3514.0,31.500000,45.666667,38.500000,9.000000,3.500000,0.0,0.166667,0.000000,0.0,...,11.833333,18.833333,7.666667,0.560847,1.333333,0.000000,15.500000,0.833333,10.000000,0
1305,212028.0,41.000000,46.000000,23.000000,6.000000,10.000000,0.0,0.000000,0.000000,0.0,...,6.000000,24.000000,11.000000,0.900000,0.000000,0.000000,3.000000,0.000000,12.000000,3
1068,141135.0,58.000000,62.000000,21.500000,11.500000,10.000000,0.0,0.000000,0.000000,0.0,...,12.500000,26.000000,11.000000,0.866667,3.000000,0.500000,7.000000,0.500000,13.500000,0
28,12790.0,45.619336,61.000000,44.097258,7.638384,2.503463,0.0,0.028571,0.022222,0.0,...,10.519913,21.911833,10.254978,0.923176,1.000000,0.000000,4.477489,0.011111,12.925253,3
165,3385.0,34.125000,105.625000,37.125000,15.125000,12.375000,0.0,0.000000,0.000000,0.0,...,9.250000,18.875000,8.875000,0.718006,9.875000,1.000000,10.375000,0.750000,12.250000,0
438,60100.0,8.857143,90.285714,19.785714,22.285714,7.000000,0.0,0.000000,0.285714,0.0,...,2.285714,12.714286,5.857143,0.240363,7.000000,0.857143,8.285714,0.000000,3.357143,3
136,41560.0,13.400000,58.600000,40.600000,4.300000,2.500000,0.0,0.000000,0.000000,0.0,...,5.800000,22.600000,2.200000,1.033333,1.000000,0.000000,7.200000,0.200000,6.600000,3
1374,198746.0,37.000000,110.000000,19.000000,27.000000,4.000000,0.0,0.000000,0.000000,0.0,...,8.000000,26.000000,12.000000,0.818182,5.000000,1.000000,7.000000,0.000000,10.000000,0


In [8]:
# Convert all columns from float to integer
f = f.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

In [9]:
f.sample(5)

,PATNO,SDMTOTAL,STAI_TOT,SFT_TOT,SCOPA_AUT_TOT,REMSLEEP_TOT,QUIP_A,QUIP_B,QUIP_C,QUIP_D,...,LNS_TOT,HVLT_TOT_Recall,HVLT_DCR_REC,HVLT_RETENTION,GDS_TOT,GDS_Depressed,ESS_TOT,ESS_Sleepy,BJLOT_TOT,COHORT
27,3067,34,57,40,12,5,0,0,0,0,...,10,19,8,0,3,0,4,0,13,0
1513,235452,61,44,28,7,5,0,0,0,0,...,12,28,11,0,0,0,4,0,15,3
239,52634,39,82,58,7,3,0,0,0,0,...,6,31,11,0,4,0,4,0,11,3
854,146935,45,47,14,7,7,0,0,0,0,...,6,20,5,0,1,0,6,0,14,3
62,3257,58,44,62,4,2,0,0,0,0,...,9,27,11,0,0,0,8,0,13,1


In [10]:
f['COHORT'].value_counts()

COHORT
0    1752
3    1737
1     357
2      81
Name: count, dtype: int64

In [11]:
X =  f.drop(['COHORT'],axis=1)
y= f['COHORT']

X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True)

In [12]:
svm = SVC(kernel="rbf", gamma="auto",probability=True)
svm.fit(X_train,y_train)
prediction = svm.predict(X_test)

accuracy_SVM = accuracy_score(prediction, y_test)
print(accuracy_SVM)

0.4480651731160896


In [13]:
dec = DecisionTreeClassifier(random_state=0)
dec.fit(X_train,y_train)
pred = dec.predict(X_test)
accuracy_DEC = accuracy_score(pred, y_test)
print(accuracy_DEC)

0.6211812627291242


In [14]:
rand = RandomForestClassifier(max_depth=15, random_state=0)
rand.fit(X_train,y_train)
rand_pred = rand.predict(X_test)
rand_accuracy = accuracy_score(rand_pred,y_test)
print(rand_accuracy)

0.714867617107943


In [15]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train,y_train)
neigh_pred = neigh.predict(X_test)
neigh_accuracy = accuracy_score(neigh_pred,y_test)
print(neigh_accuracy)

0.6313645621181263
